In [1]:
from hamiltonian import HamiltonianSmall, Hamiltonian
from var_opt import objective_diagonal

import var_opt_lagrange

# Lagrange by Rudy

See Rudy's note on optimizing the variance with Lagrange multiplier method

In [2]:
lih_small = HamiltonianSmall('LiH', 1.5) # this encoding uses reduction techniques
beh2_small = HamiltonianSmall('BeH2', 1.3) # this encoding uses reduction techniques

# 4 qubits
h2_jw_4 = Hamiltonian('H2_STO3g_4qubits', 'jw')
h2_parity_4 = Hamiltonian('H2_STO3g_4qubits', 'parity')
h2_bk_4 = Hamiltonian('H2_STO3g_4qubits', 'bk')

# 8 qubits
h2_jw = Hamiltonian('H2_6-31G_8qubits', 'jw')
h2_parity = Hamiltonian('H2_6-31G_8qubits', 'parity')
h2_bk = Hamiltonian('H2_6-31G_8qubits', 'bk')

In [3]:
hamiltonians = {"lih_small": lih_small,
                "beh2_small": beh2_small,
                "h2_jw_4": h2_jw_4,
                "h2_parity_4": h2_parity_4,
                "h2_bk_4": h2_bk_4,
                "h2_jw": h2_jw,
                "h2_parity": h2_parity,
                "h2_bk": h2_bk}           

An example showing how to call this from `Hamiltonian.PauliRep.local_dists_optimal`

In [4]:
from numpy.random import rand
import numpy as np

np.random.seed(10)

def isProbability(betas, tol=1.0e-5):
    """
        Check if betas are probabilities
    """
    for k,v in betas.items():
        for _v in v:
            if _v < 0:
                return False
        if np.abs(np.sum(v) - 1.0) > tol:
            print("\tErr:", np.abs(np.sum(v) - 1.0) )
            return False
    return True

def get_rand_betas(nQubits):
    """
        generate random initial points to search for betas
    """
    betas = {}
    for n in range(nQubits):
        betas[n] = rand(3)
        betas[n] = betas[n]/np.sum(betas[n])
    return betas

In [5]:
ham = h2_jw_4
energy, state = ham.pauli_rep.ground()

objective = 'diagonal'
method = 'lagrange'

β_optimal_diagonal = ham.pauli_rep.local_dists_optimal(objective, method)

var = ham.pauli_rep.variance_local(energy, state, β_optimal_diagonal)
print("Diagonal:", var)
#print("\tBetas:", β_optimal_diagonal )

objective = "mixed"
bits_HF = ham.read_bitstring_HF()
ntrials = 10
for _ in range(ntrials): #because we do not have convexity, we perform several trials to look for optimal betas
    if _ == 0:
        beta_init = β_optimal_diagonal
    else:
        beta_init = get_rand_betas(ham.pauli_rep.num_qubits)
    β_optimal_mixed = ham.pauli_rep.local_dists_optimal(objective, method, bitstring_HF=bits_HF, β_initial=beta_init)
    if not isProbability(β_optimal_mixed ):
        print("WARNING: not probability:", β_optimal_mixed)
    var = ham.pauli_rep.variance_local(energy, state, β_optimal_mixed)
    print("Trial:", _, "Mixed:", var)
    print("\tBetas:", β_optimal_mixed)
    print("\n")
    #print("\tBetas:", β_optimal_diagonal)

Diagonal: 1.8555830498849546
Trial: 0 Mixed: 1.8546562200043257
	Betas: {0: [0.30257725605404756, 0.30257725605404756, 0.3948454878919052], 1: [0.3145814335670787, 0.3145814335670787, 0.37083713286584297], 2: [0.30257725605404756, 0.30257725605404756, 0.3948454878919052], 3: [0.3145814335670787, 0.3145814335670787, 0.3708371328658429]}


Trial: 1 Mixed: 1.85465625858608
	Betas: {0: [0.3025992578644691, 0.30253491957050804, 0.3948658225650232], 1: [0.3145592428094682, 0.3146261385978969, 0.3708146185926352], 2: [0.3025426516043169, 0.3025915288373723, 0.39486581955831096], 3: [0.31461589474987073, 0.3145650756870394, 0.3708190295630901]}


Trial: 2 Mixed: 1.85465625906689
	Betas: {0: [0.30253772809709073, 0.30259537691242716, 0.3948668949904824], 1: [0.31462256149399087, 0.31456262081819386, 0.3708148176878154], 2: [0.30259572323224004, 0.3025389395902336, 0.39486533717752675], 3: [0.31456135941759444, 0.3146203990915848, 0.37081824149082104]}


Trial: 3 Mixed: 1.8546562606384822
	Betas

In [6]:
ham = h2_jw
energy, state = ham.pauli_rep.ground()

objective = 'diagonal'
method = 'lagrange'

β_optimal_diagonal = ham.pauli_rep.local_dists_optimal(objective, method)

var = ham.pauli_rep.variance_local(energy, state, β_optimal_diagonal)
print("Diagonal:", var)
#print("\tBetas:", β_optimal_diagonal )

objective = "mixed"
bits_HF = ham.read_bitstring_HF()
ntrials = 10
for _ in range(ntrials): #because we do not have convexity, we perform several trials to look for optimal betas
    if _ == 0:
        beta_init = β_optimal_diagonal
    else:
        beta_init = get_rand_betas(ham.pauli_rep.num_qubits)
    β_optimal_mixed = ham.pauli_rep.local_dists_optimal(objective, method, bitstring_HF=bits_HF, β_initial=beta_init)
    if not isProbability(β_optimal_mixed ):
        print("WARNING: not probability:", β_optimal_mixed)
    var = ham.pauli_rep.variance_local(energy, state, β_optimal_mixed)
    print("Trial:", _, "Mixed:", var)
    print("\n")
    #print("\tBetas:", β_optimal_diagonal)

Diagonal: 17.741947811137187
Trial: 0 Mixed: 17.45718728202788


Trial: 1 Mixed: 17.457191336127558


Trial: 2 Mixed: 17.457191399347536


Trial: 3 Mixed: 17.457192143682654


Trial: 4 Mixed: 17.45719124671826


Trial: 5 Mixed: 17.457190616009115


Trial: 6 Mixed: 17.457191113903882


Trial: 7 Mixed: 17.45719172291433


Trial: 8 Mixed: 17.45719138908455


Trial: 9 Mixed: 17.45719161926231




some of this code has been used to build into the function `find_optimal_beta_lagrange` which is called in the above `local_dists_optimal`

In [ ]:
def test_iterative_updates(tol=1.0e-5):
    for name, ham in hamiltonians.items():
        
        print("Molecule=", name, "------------------")
        num_qubits = ham.pauli_rep.num_qubits
        energy, state = ham.pauli_rep.ground()
        # compare lagrange method with scipy method
        β_opt = ham.pauli_rep.local_dists_optimal('diagonal', 'scipy')
        # the cost function's value using scipy reference
        objective = objective_diagonal(ham.pauli_rep.dic_tf, β_opt) 
         
        # THIS IS THE EXPERIMENT WITH ITERATIVE UPDATES TO FIND BETAS 
        tol = 1.0e-5
        iter = 0
        β_old = None
        while True and iter < 1000:
            β_new, error = var_opt_lagrange.update_betas(ham.pauli_rep.dic_tf, num_qubits, β_old)
            β_old = β_new
            print("\t", iter, error)
            iter += 1
            if error < tol:
                print("MOLECULE", name, "CONVERGE with error = ", error)
                break

        print("Distance from true values:", var_opt_lagrange.distance(β_new, β_opt)) 

In [ ]:
test_iterative_updates()

In [ ]:
from numpy.random import rand
sum(rand(3))

Diagonal: 1.8555830498849475
	Betas: {0: [0.3021412813203303, 0.3021412813203303, 0.3957174373593399], 1: [0.31122347161788594, 0.31122347161788594, 0.3775530567642284], 2: [0.3021412813203302, 0.3021412813203302, 0.3957174373593399], 3: [0.31122347161788594, 0.31122347161788594, 0.3775530567642283]}
Iter: 1000 Error: 0.003941286587053889
Iter: 2000 Error: 0.016124144013247854
Iter: 3000 Error: 0.0013773377918440202
Iter: 4000 Error: 0.0005625655094954146
Iter: 5000 Error: 0.16745325997376168
Iter: 6000 Error: 0.0037267060386065175
Iter: 7000 Error: 0.0010022833021886052
Iter: 8000 Error: 0.014058898602006802
Iter: 9000 Error: 0.009958090528469144
Iter: 10000 Error: 0.00512673159169223
Trial: 0 Mixed: 2.543555046805095
Iter: 1000 Error: 0.003585242723138465
Iter: 2000 Error: 0.0006914648612049422
Iter: 3000 Error: 0.019647934158187063
Iter: 4000 Error: 0.14023438689371548
Iter: 5000 Error: 0.06919661838525358
Iter: 6000 Error: 0.41345990820078754
Iter: 7000 Error: 0.002178859090536059
Iter: 8000 Error: 0.0011179009710741525
Iter: 9000 Error: 0.0007945791951767325
Iter: 10000 Error: 0.0005672610616211887
Trial: 1 Mixed: 1.8039168252108944
Iter: 1000 Error: 0.0007137036093656794
Iter: 2000 Error: 0.009730865955007655
Iter: 3000 Error: 0.002560873734534799
Iter: 4000 Error: 0.0016906855997481894
Iter: 5000 Error: 0.0028324585421971834
Iter: 6000 Error: 0.000596670549329644
Iter: 7000 Error: 0.001221240154815479
Iter: 8000 Error: 0.001802810005417958
Iter: 9000 Error: 0.04876421820803623
Iter: 10000 Error: 0.05962252126462637
Trial: 2 Mixed: -0.47124594133782327
Iter: 1000 Error: 0.01662016838850315
Iter: 2000 Error: 0.0005615385541332986
Iter: 3000 Error: 0.0006200138185703638
Iter: 4000 Error: 0.0030712525442489155
Iter: 5000 Error: 0.0006403161420553304
Iter: 6000 Error: 0.0020661182018662048
Iter: 7000 Error: 0.0012115793422569293
Iter: 8000 Error: 0.0006234949648077488
Iter: 9000 Error: 0.003875781025590037
Iter: 10000 Error: 0.0014577184675726414
Trial: 3 Mixed: 1.789360660075212
Iter: 1000 Error: 0.012008335525235989
Iter: 2000 Error: 0.0014364311916412267
Iter: 3000 Error: 0.012521399454954028
Iter: 4000 Error: 0.0006407194758113205
Iter: 5000 Error: 0.0005604531878207529
Iter: 6000 Error: 0.02229275159923158
Iter: 7000 Error: 0.000584561049029949
Iter: 8000 Error: 0.031259360692849535
Iter: 9000 Error: 0.029920644521126277
Iter: 10000 Error: 0.9278196896893477
Trial: 4 Mixed: 0.025781631119184745
Iter: 1000 Error: 0.0007858044796963242
Iter: 2000 Error: 0.009216318227607826
Iter: 3000 Error: 0.049915401198954386
Iter: 4000 Error: 0.0016604131216975604
Iter: 5000 Error: 0.0007155061726851822
Iter: 6000 Error: 0.009662142792307728
Iter: 7000 Error: 0.003983315139492419
Iter: 8000 Error: 0.004041784439071653
Iter: 9000 Error: 0.010462345963707391
Iter: 10000 Error: 0.0025057640738637074
Trial: 5 Mixed: 1.8206407260865012
Iter: 1000 Error: 0.0008473848239091968
Iter: 2000 Error: 0.008947063585779846
Iter: 3000 Error: 0.009341401079622134
Iter: 4000 Error: 0.005610122767576232
Iter: 5000 Error: 0.0016286986829575029
Iter: 6000 Error: 0.0032632002012440535
Iter: 7000 Error: 0.0007658496127080361
Iter: 8000 Error: 0.028318429840387887
Iter: 9000 Error: 0.0028227880340533054
Iter: 10000 Error: 0.0012243209401208493
Trial: 6 Mixed: 2.074844592316016
Iter: 1000 Error: 0.026866154360524152
Iter: 2000 Error: 0.02076632415773604
Iter: 3000 Error: 0.0018929669909757915
Iter: 4000 Error: 0.01282794114082728
Iter: 5000 Error: 0.028568928618714826
Iter: 6000 Error: 0.003688059707749385
Iter: 7000 Error: 0.002081871310478431
Iter: 8000 Error: 0.0349884222755818
Iter: 9000 Error: 0.00133191439620874
Iter: 10000 Error: 0.0035533288922693983
Trial: 7 Mixed: 2.044075955500458
Iter: 1000 Error: 0.02843167712425945
Iter: 2000 Error: 0.006132341374506825
Iter: 3000 Error: 0.0008297732679764637
Iter: 4000 Error: 0.003963053241234429
Iter: 5000 Error: 0.003227748616219524
Iter: 6000 Error: 0.0008387669387437423
Iter: 7000 Error: 0.0005910310942863078
Iter: 8000 Error: 0.013268481342953123
Iter: 9000 Error: 0.0008816996731647081
Iter: 10000 Error: 0.0007684936682637051
Trial: 8 Mixed: 1.9239192804531768
Iter: 1000 Error: 0.02845800766269487
Iter: 2000 Error: 0.13189651427015855
Iter: 3000 Error: 0.009206329322852825
Iter: 4000 Error: 0.0044196062182191445
Iter: 5000 Error: 0.0008849940467915989
Iter: 6000 Error: 0.01350533042122539
Iter: 7000 Error: 0.0006028758341822034
Iter: 8000 Error: 0.03208253244348048
Iter: 9000 Error: 0.0014826342748445608
Iter: 10000 Error: 0.01661578981761769
Trial: 9 Mixed: 8.154931496417378
